<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/MySQLv8_CTE_Recursion_Windows_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install, Connect

## Local MySQL 

In [1]:
!apt update > /dev/null
!apt install -y  mysql-server > /dev/null
# local MySQL v 5.6 is NOT STARTED
!/etc/init.d/mysql restart






 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [2]:
!mysql --version

mysql  Ver 8.0.31-0ubuntu0.20.04.2 for Linux on x86_64 ((Ubuntu))


In [ ]:
#!mysql -e 'create database praxisDB'
#!mysql -e 'show databases'

## Set Public Credentials for remote MySQL v8 server

In [ ]:
# enter credentials here
#hostName = 'remotemysql.com'   # place your own credentials here
#userName = 'yourUserName'      # place your own credentials here
#passWord = 'yourPassWord'      # place your own credentials here
#dbName =  'yourDBname'         # place your own credentials here

#print(hostName,userName,dbName)

## GET Private Credentials for remote MySQL v8 server

In [ ]:
#connect to G-Drive
from google.colab import drive
drive.mount('/content/drive')
# copy credentials file
#!ls /content/drive/'My Drive'/Praxis/WebCredentials
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/sqlCredentials_020221.py .
!cp /content/drive/'My Drive'/Praxis/WebCredentials/cleverCloud.py credentials.py
from credentials import hostName,userName,passWord,dbName

#for the sake of privacy
#the following credentials need to be stored in a text file called sqlCredentials.py
#in the format given below
#in the Colab VM


print(hostName,userName,dbName)

Mounted at /content/drive
bq3ojarbqgmkc7prtopv-mysql.services.clever-cloud.com uxnhrnbgcu47thlz bq3ojarbqgmkc7prtopv


In [ ]:
#NOTE SECURITY WARNING
#Passing passwords in the command line is inherently risky and SHOULD NOT BE USED for any non-trivial work
#---------------------------------------------------------------------
#!mysql -h $hostName -P 3306 -u $userName -p$passWord $dbName -e 'show tables'

# Pythons Functions for SQL commands

In [3]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb

In [8]:
#To run any non-SELECT SQL command
def runCMD (loc, DDL):
    if loc == 'remoteDB':
        DBConn = MySQLdb.connect(hostName,userName,passWord,dbName)
    elif loc == 'localDB':
        DBConn = MySQLdb.connect(db='classicmodels')
        #DBConn = MySQLdb.connect(db='praxisDB')
    else :
        print('location must be "remoteDB or "localDB"')
        return
    
    myCursor = DBConn.cursor()
    retcode = myCursor.execute(DDL) 
    print (retcode)
    DBConn.commit()
    DBConn.close()

#To run any SELECT SQL command
def runSELECT (loc, CMD):
    if loc == 'remoteDB':
        DBConn = MySQLdb.connect(hostName,userName,passWord,dbName)
    elif loc == 'localDB':
        DBConn = MySQLdb.connect(db='classicmodels')
        #DBConn = MySQLdb.connect(db='praxisDB')
    else :
        print('location must be "remoteDB or "localDB"')
        return
    
    df_mysql = pd.read_sql(CMD, con=DBConn)    
    DBConn.close()
    return df_mysql

In [4]:
# Set the location to whichever Database you need to access
#
# --------------------------------
#
location = 'localDB'
#location = 'remoteDB'

#Create, Load Classic Models
Create Tables
https://www.mysqltutorial.org/mysql-sample-database.aspx <br>
DO NOT RECREATE TABLES if the already exist on remote system

In [5]:
!wget -q https://github.com/Praxis-QR/RDWH/raw/main/data/mysqlsampledatabase.zip
!unzip mysqlsampledatabase.zip
#!cat /content/mysqlsampledatabase.sql
!head /content/mysqlsampledatabase.sql

Archive:  mysqlsampledatabase.zip
  inflating: mysqlsampledatabase.sql  
/*
*********************************************************************
http://www.mysqltutorial.org
*********************************************************************
Name: MySQL Sample Database classicmodels
Link: http://www.mysqltutorial.org/mysql-sample-database.aspx
Version 3.1
+ changed data type from DOUBLE to DECIMAL for amount columns
Version 3.0
+ changed DATETIME to DATE for some colunmns


## Only for Remote operations
ignore next three cells for local operations

In [ ]:
#locate lines with the string 'classicmodels'
!grep 'classicmodels' mysqlsampledatabase.sql

Name: MySQL Sample Database classicmodels
CREATE DATABASE /*!32312 IF NOT EXISTS*/`classicmodels` /*!40100 DEFAULT CHARACTER SET latin1 */;
USE `classicmodels`;


In [ ]:
# remove the lines with the string 'classicmodels'
!sed -i '/classicmodels/d' mysqlsampledatabase.sql

In [ ]:
#locate lines with the string 'classicmodels'
!grep 'classicmodels' mysqlsampledatabase.sql

In [6]:
#https://www.mysqltutorial.org/how-to-load-sample-database-into-mysql-database-server.aspx
#!mysql -h $hostName -P 3306 -u $userName -p$passWord $dbName --table < mysqlsampledatabase.sql
!mysql  < mysqlsampledatabase.sql
#New database classicmodels has been created


In [7]:
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| classicmodels      |
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+


In [9]:
#!mysql -h $hostName -P 3306 -u $userName -p$passWord $dbName -e 'show tables'
runSELECT(location, 'show tables')

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


# CTE - Common Table Extensions

In [10]:
sqlString = "WITH customers_in_usa AS ( \
    SELECT customerName, state \
    FROM customers \
    WHERE country = 'USA' \
) SELECT customerName \
 FROM customers_in_usa \
 WHERE state = 'CA' \
 ORDER BY customerName"

runSELECT (location, sqlString)

,customerName
0,Boards & Toys Co.
1,Collectable Mini Designs Co.
2,Corporate Gift Ideas Co.
3,"Men 'R' US Retailers, Ltd."
4,Mini Gifts Distributors Ltd.
5,Mini Wheels Co.
6,Signal Collectibles Ltd.
7,Technics Stores Inc.
8,The Sharp Gifts Warehouse
9,Toys4GrownUps.com


In [11]:
sqlString = "\
WITH topsales2003 AS (\
    SELECT \
        salesRepEmployeeNumber employeeNumber,\
        SUM(quantityOrdered * priceEach) sales\
    FROM\
        orders\
            INNER JOIN\
        orderdetails USING (orderNumber)\
            INNER JOIN\
        customers USING (customerNumber)\
    WHERE\
        YEAR(shippedDate) = 2003\
            AND status = 'Shipped'\
    GROUP BY salesRepEmployeeNumber\
    ORDER BY sales DESC\
    LIMIT 5\
)\
SELECT \
    employees.employeeNumber, firstName, lastName, sales\
 FROM \
     topsales2003, employees where employees.employeeNumber = topsales2003.employeeNumber"


runSELECT (location, sqlString)

,employeeNumber,firstName,lastName,sales
0,1165,Leslie,Jennings,413219.85
1,1370,Gerard,Hernandez,295246.44
2,1401,Pamela,Castillo,289982.88
3,1621,Mami,Nishi,267249.40
4,1501,Larry,Bott,261536.95


In [12]:
sqlString = "\
WITH salesrep AS ( \
    SELECT \
        employeeNumber, \
        CONCAT(firstName, ' ', lastName) AS salesrepName \
    FROM \
        employees \
    WHERE \
        jobTitle = 'Sales Rep' \
), \
customer_salesrep AS ( \
    SELECT \
        customerName, salesrepName \
    FROM \
        customers \
            INNER JOIN \
        salesrep ON employeeNumber = salesrepEmployeeNumber \
) \
SELECT \
    * \
FROM \
    customer_salesrep \
ORDER BY customerName"

runSELECT (location, sqlString)

,customerName,salesrepName
0,Alpha Cognac,Gerard Hernandez
1,American Souvenirs Inc,Foon Yue Tseng
2,Amica Models & Co.,Pamela Castillo
3,"Anna's Decorations, Ltd",Andy Fixter
4,Atelier graphique,Gerard Hernandez
...,...,...
95,"UK Collectables, Ltd.",Larry Bott
96,"Vida Sport, Ltd",Martin Gerard
97,Vitachrome Inc.,Foon Yue Tseng
98,"Volvo Model Replicas, Co",Barry Jones


# Recursion

In [13]:
sqlString = "\
 WITH RECURSIVE cte_count (n) \
AS ( \
      SELECT 1 \
      UNION ALL \
      SELECT n + 1 \
      FROM cte_count \
      WHERE n < 3 \
    ) \
SELECT n \
FROM cte_count"


runSELECT (location, sqlString)

,n
0,1
1,2
2,3


In [14]:
sqlString = "\
WITH RECURSIVE employee_paths AS \
  ( SELECT employeeNumber, \
           reportsTo managerNumber, \
           officeCode, \
           1 lvl \
   FROM employees \
   WHERE reportsTo IS NULL \
     UNION ALL \
     SELECT e.employeeNumber, \
            e.reportsTo, \
            e.officeCode, \
            lvl+1 \
     FROM employees e \
     INNER JOIN employee_paths ep ON ep.employeeNumber = e.reportsTo ) \
SELECT employeeNumber, \
       managerNumber, \
       lvl, \
       city \
FROM employee_paths ep \
INNER JOIN offices o USING (officeCode) \
ORDER BY lvl, city"


runSELECT (location, sqlString)

,employeeNumber,managerNumber,lvl,city
0,1002,NaN,1,San Francisco
1,1056,1002.0,2,San Francisco
2,1076,1002.0,2,San Francisco
3,1102,1056.0,3,Paris
4,1143,1056.0,3,San Francisco
5,1088,1056.0,3,Sydney
6,1621,1056.0,3,Tokyo
7,1188,1143.0,4,Boston
8,1216,1143.0,4,Boston
9,1504,1102.0,4,London


#Window Functions

In [15]:
runSELECT(location, 'select * from payments')

,customerNumber,checkNumber,paymentDate,amount
0,103,HQ336336,2004-10-19,6066.78
1,103,JM555205,2003-06-05,14571.44
2,103,OM314933,2004-12-18,1676.14
3,112,BO864823,2004-12-17,14191.12
4,112,HQ55022,2003-06-06,32641.98
...,...,...,...,...
268,495,BH167026,2003-12-26,59265.14
269,495,FN155234,2004-05-14,6276.60
270,496,EU531600,2005-05-25,30253.75
271,496,MB342426,2003-07-16,32077.44


In [16]:
#basic group by
sqlString = 'select customerNumber, count(checkNumber) cheques, sum(amount) total from payments group by customerNumber'
runSELECT(location,sqlString)

,customerNumber,cheques,total
0,103,3,22314.36
1,112,3,80180.98
2,114,4,180585.07
3,119,3,116949.68
4,121,4,104224.79
...,...,...,...
93,486,3,77726.59
94,487,2,42570.37
95,489,2,29586.15
96,495,2,65541.74


In [17]:
#SQL Windows
sqlString = 'select customerNumber, checkNumber, paymentDate, amount,   sum(amount) over (partition by customerNumber) total from payments '
runSELECT(location,sqlString)

,customerNumber,checkNumber,paymentDate,amount,total
0,103,HQ336336,2004-10-19,6066.78,22314.36
1,103,JM555205,2003-06-05,14571.44,22314.36
2,103,OM314933,2004-12-18,1676.14,22314.36
3,112,BO864823,2004-12-17,14191.12,80180.98
4,112,HQ55022,2003-06-06,32641.98,80180.98
...,...,...,...,...,...
268,495,BH167026,2003-12-26,59265.14,65541.74
269,495,FN155234,2004-05-14,6276.60,65541.74
270,496,EU531600,2005-05-25,30253.75,114497.19
271,496,MB342426,2003-07-16,32077.44,114497.19


In [18]:
sqlString = 'select customerNumber, checkNumber, sum(amount) over (partition by customerNumber) total ,count(checkNumber) over (partition by customerNumber) cheques from payments '
runSELECT(location,sqlString)

,customerNumber,checkNumber,total,cheques
0,103,HQ336336,22314.36,3
1,103,JM555205,22314.36,3
2,103,OM314933,22314.36,3
3,112,BO864823,80180.98,3
4,112,HQ55022,80180.98,3
...,...,...,...,...
268,495,BH167026,65541.74,2
269,495,FN155234,65541.74,2
270,496,EU531600,114497.19,3
271,496,MB342426,114497.19,3


In [19]:
sqlString = '\
select customerNumber, sum(amount) over w as total , count(checkNumber) over w as checks \
from payments \
window w as (partition by customerNumber)'
runSELECT(location,sqlString)

,customerNumber,total,checks
0,103,22314.36,3
1,103,22314.36,3
2,103,22314.36,3
3,112,80180.98,3
4,112,80180.98,3
...,...,...,...
268,495,65541.74,2
269,495,65541.74,2
270,496,114497.19,3
271,496,114497.19,3


In [20]:
runSELECT(location,'select * from orderdetails limit 30')

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4
5,10101,S18_2795,26,167.06,1
6,10101,S24_1937,45,32.53,3
7,10101,S24_2022,46,44.35,2
8,10102,S18_1342,39,95.55,2
9,10102,S18_1367,41,43.13,1


In [21]:
sqlString = 'select orderNumber, productCode, sum(quantityOrdered) from orderdetails group by orderNumber, productCode'
runSELECT(location, sqlString)

,orderNumber,productCode,sum(quantityOrdered)
0,10100,S18_1749,30.0
1,10100,S18_2248,50.0
2,10100,S18_4409,22.0
3,10100,S24_3969,49.0
4,10101,S18_2325,25.0
...,...,...,...
2991,10425,S24_2300,49.0
2992,10425,S24_2840,31.0
2993,10425,S32_1268,41.0
2994,10425,S32_2509,11.0


In [22]:
sqlString = 'select productCode, sum(quantityOrdered) from orderdetails group by productCode'
runSELECT(location, sqlString)

,productCode,sum(quantityOrdered)
0,S10_1678,1057.0
1,S10_1949,961.0
2,S10_2016,999.0
3,S10_4698,985.0
4,S10_4757,1030.0
...,...,...
104,S700_3505,952.0
105,S700_3962,896.0
106,S700_4002,1085.0
107,S72_1253,960.0


In [23]:
sqlString = 'select orderNumber,  sum(quantityOrdered * priceEach) orderValue from orderdetails group by orderNumber'
runSELECT(location, sqlString)

,orderNumber,orderValue
0,10100,10223.83
1,10101,10549.01
2,10102,5494.78
3,10103,50218.95
4,10104,40206.20
...,...,...
321,10421,7639.10
322,10422,5849.44
323,10423,8597.73
324,10424,29310.30


In [24]:
location = 'funny'
runSELECT(location,'select * from Emp')

location must be "remoteDB or "localDB"


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)